In [1]:
import time
import json
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

from src.data.dataset_info import datasets
from src.models import MyCNN, MyLSTM, MyGRU, MyDenseNN
# from src.models.dense_nn import  MyDenseNN

multi_class = True
with_network_features = False

with_sort_timestamp = True
sequence_length = 3
with_cross_validation = True
cross_validation_splits_num = 5

dataset = datasets[0]
name = dataset.name
print("dataset: {}".format(name))
path = "./datasets/preprocessed/{}.pkl".format(name)
# graph_path = "./datasets/preprocessed/graph_{}.gexf".format(name)
df = pd.read_pickle(path)

dataset: cic_ton_iot


In [2]:
input_dim = df.shape[1] - len(dataset.drop_columns) - len(dataset.weak_columns) - 1  # for the label_column

if not with_network_features:
    input_dim = input_dim - len(dataset.network_features)

num_classes = 2
if multi_class:
    num_classes = len(df["Attack"].unique())

num_epochs = 50
    
dropped_columns = dataset.drop_columns
dataset_name = dataset.name
print(f"==>> dataset_name: {dataset_name}")

==>> dataset_name: cic_ton_iot


In [3]:


def create_mixed_samples(data, labels, sequence_length):
    """
    Generate both sequential and non-sequential samples for training various models.

    Parameters:
    - data: The time series data as a 1D numpy array.
    - labels: The corresponding labels for each data point.
    - sequence_length: The length of each sequential sample.

    Returns:
    - A tuple containing three numpy arrays:
      1. Sequential samples: 3D array representing sequential samples.
      2. Non-sequential samples: 2D array representing non-sequential samples.
      3. Associated labels: 1D array containing labels corresponding to each sample.
    """
    sequential_samples = []
    non_sequential_samples = []
    associated_labels = []
    data_len = len(data)

    for i in range(data_len - sequence_length + 1):
        # Sequential samples
        seq = data[i:i+sequence_length]
        sequential_samples.append(seq)
        associated_labels.append(labels[i + sequence_length - 1])

        # Non-sequential samples (individual data points)
        non_seq = data[i + sequence_length - 1]
        non_sequential_samples.append(non_seq)

    return (
        np.array(sequential_samples),
        np.array(non_sequential_samples),
        np.array(associated_labels)
    )



In [4]:
nf = []
if with_network_features:
    nf = dataset.network_features

models = [
    MyDenseNN(
        input_dim=input_dim,
        dataset_name=dataset_name,
        num_classes=num_classes,
        multi_class=multi_class,
        network_features=nf,
        epochs=num_epochs,
        batch_size=256,
        early_stop_patience=10
    ),
    # MyCNN(
    #     input_dim=input_dim,
    #     dataset_name=dataset_name,
    #     num_classes=num_classes,
    #     multi_class=multi_class,
    #     network_features=nf,
    #     epochs=num_epochs,
    #     batch_size=256,
        # early_stop_patience=10,
    # ),
    # MyLSTM(
    #     sequence_length=sequence_length,
    #     input_dim=input_dim,
    #     dataset_name=dataset_name,
    #     num_classes=num_classes,
    #     multi_class=multi_class,
    #     network_features=nf,
    #     use_generator=True,
    #     epochs=num_epochs,
    #     batch_size=256,,
        # early_stop_patience=10,
    # ),
    # MyGRU(
    #     sequence_length=sequence_length,
    #     input_dim=input_dim,
    #     dataset_name=dataset_name,
    #     num_classes=num_classes,
    #     multi_class=multi_class,
    #     network_features=nf,
    #     use_generator=True,
    #     epochs=num_epochs,
    #     batch_size=256,,
        # early_stop_patience=10,
    # )
]

In [5]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "stratified k-fold cross validation - manual sequences"
results["multi_class"] = multi_class
results["with_sort_timestamp"] = with_sort_timestamp
results["sequence_length"] = sequence_length
results["with_cross_validation"] = with_cross_validation
results["cross_validation_splits_num"] = cross_validation_splits_num
results["with_network_features"] = with_network_features
results["network_features"] = dataset.cn_measures

results["dataset_name"] = dataset_name
results["input_dim"] = input_dim
results["dropped_columns"] = dropped_columns
results["num_dropped_columns"] = len(dropped_columns)

results["models"] = {}
results["average_acc"] = {}
results["average"] = {}

In [6]:

if with_sort_timestamp:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)

    # sorted_grouped = df.sort_values(
    #     self.datasetInfo.timestamp_col).groupby(self.datasetInfo.src_ip_col)

    # sorted_grouped = df.sort_values(
    #     self.datasetInfo.timestamp_col).groupby(self.datasetInfo.dst_ip_col)

    # df = pd.concat([group for _, group in sorted_grouped])

    df.sort_values(dataset.timestamp_col, inplace= True)

labels_names = {0: "benign", 1: "attack"}
if multi_class:
    fac = pd.factorize(df[dataset.class_col])
    labels_names = {index: value for index, value in enumerate(fac[1])}
    print(f"==>> labels_names: {labels_names}")
    df[dataset.label_col] = fac[0]  # type: ignore


df.drop(dataset.drop_columns, axis=1, inplace=True)
df.drop(dataset.weak_columns, axis=1, inplace=True)

if not with_network_features:
    df = df.drop(dataset.network_features, axis=1)

df.reset_index(drop=True, inplace=True)


==>> labels_names: {0: 'Benign', 1: 'xss', 2: 'password', 3: 'scanning', 4: 'injection', 5: 'ransomware', 6: 'backdoor', 7: 'mitm', 8: 'ddos', 9: 'dos'}


In [7]:
df.head()

,Protocol,Tot Fwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Min,Bwd Pkt Len Min,Flow Byts/s,Flow IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Std,...,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Max,Active Min,Idle Std,Idle Max,Idle Min,Label
0,0.0,3.0,0.0,0.0,0.0,0.0,6.306497e+07,102196170.0,13008834.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.973171e+14,1.554199e+15,1.300883e+07,0
1,0.0,3.0,0.0,0.0,0.0,0.0,1.373953e+06,3991265.0,2048202.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.554199e+15,1.554199e+15,0
2,0.0,3.0,0.0,0.0,0.0,0.0,1.564027e+07,43827824.0,21709138.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.973172e+14,1.554199e+15,2.170914e+07,0
3,0.0,3.0,0.0,0.0,0.0,0.0,4.231300e+07,75008320.0,15168707.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.973172e+14,1.554199e+15,1.516871e+07,0
4,0.0,3.0,0.0,0.0,0.0,0.0,1.880029e+06,57137762.0,54478999.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.973172e+14,1.554199e+15,5.447900e+07,0


In [8]:
labels = df['Label'].to_numpy()
df = df.drop([dataset.label_col], axis=1).to_numpy()

isThereLSTM = False
if isThereLSTM:
    df, df_non_seq, labels = create_mixed_samples(
        df, labels, sequence_length)
else:
    df_non_seq = df

In [9]:

skf = StratifiedKFold(
    n_splits=cross_validation_splits_num, shuffle=True, random_state=24)
i = 0
for train_index, test_index in skf.split(df, labels):
    training_labels = labels[train_index]
    print(f"==>> train_index: {train_index}")
    print(f"==>> training_labels: {training_labels.shape}")
    testing_labels = labels[test_index]
    print(f"==>> test_index: {test_index}")
    print(f"==>> testing_labels: {testing_labels.shape}")

    i += 1
    print("fold: {}".format(i))
    # print("train_index: {}".format(train_index))
    print("=====================================")
    print("=====================================")
    # print("fold: {}/{}".format(i, len(list_of_dfs)))
    print("fold: {}/{}".format(i, cross_validation_splits_num))

    for model in models:
        print("training: {}".format(model.model_name()))
        print("sequential: {}".format(model.sequential))

        if model.sequential:
            training = df[train_index]
            testing = df[test_index]
        else:
            training = df_non_seq[train_index]
            testing = df_non_seq[test_index]
            
            # scaler = MinMaxScaler()
            # training = scaler.fit_transform(training)
            # testing = scaler.transform(testing)

        model.build()
        model.train(training,
                    training_labels)  # type: ignore
        predictions, prediction_time = model.predict(
            testing)  # type: ignore
        model_name, scores, class_report = model.evaluate(  # type: ignore
            predictions,
            testing_labels,
            prediction_time
        )
        scores["fold"] = i
        if i == 1:
            results["models"][model_name] = {}
            results["models"][model_name]["scores"] = [scores]
            results["models"][model_name]["class_report"] = [class_report]
        else:
            results["models"][model_name]["scores"].append(scores)
            results["models"][model_name]["class_report"].append(
                class_report)
        # results[str(i) + model_name] = scores
        print("{}: {}".format(model_name, scores))

    for model in models:
        model_name = model.model_name()
        average_acc = 0
        average_recall = 0
        average_precision = 0
        average_f1s = 0
        average_FPR = 0
        average_FNR = 0
        for result in results["models"][model_name]["scores"]:  # type: ignore
            average_acc += result["accuracy"]
            average_recall += result["recall"]
            average_precision += result["precision"]
            average_f1s += result["f1s"]
            average_FPR += result["FPR"]
            average_FNR += result["FNR"]
        average_acc = average_acc / i
        average_recall = average_recall / i
        average_precision = average_precision / i
        average_f1s = average_f1s / i
        average_FPR = average_FPR / i
        average_FNR = average_FNR / i
        if i == 1:
            results["models"][model_name]["average"] = [
                {
                    "average_acc": average_acc,
                    "average_recall": average_recall,
                    "average_precision": average_precision,
                    "average_f1s": average_f1s,
                    "average_FPR": average_FPR,
                    "average_FNR": average_FNR,
                    "fold": i
                }
            ]
            results["average_acc"][model_name] = average_acc
            results["average"][model_name] = {
                "average_acc": average_acc,
                "average_recall": average_recall,
                "average_precision": average_precision,
                "average_f1s": average_f1s,
                "average_FPR": average_FPR,
                "average_FNR": average_FNR
            }
        else:
            results["models"][model_name]["average"].append(
                {
                    "average_acc": average_acc,
                    "average_recall": average_recall,
                    "average_precision": average_precision,
                    "average_f1s": average_f1s,
                    "average_FPR": average_FPR,
                    "average_FNR": average_FNR,
                    "fold": i
                })
            results["average_acc"][model_name] = average_acc
            results["average"][model_name] = {
                "average_acc": average_acc,
                "average_recall": average_recall,
                "average_precision": average_precision,
                "average_f1s": average_f1s,
                "average_FPR": average_FPR,
                "average_FNR": average_FNR
            }
        print("{} average accuracy: {}".format(model_name, average_acc))

results["endtime"] = time.strftime("%Y:%m:%d-%H:%M:%S")

print(f"==>> results: {results}")


==>> train_index: [      0       1       2 ... 5350579 5350580 5350581]
==>> training_labels: (4280466,)
==>> test_index: [     10      23      28 ... 5350573 5350578 5350582]
==>> testing_labels: (1070117,)
fold: 1
fold: 1/5
training: dense_nn mc 
sequential: False
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 37)               75        
 n)                                                              
                                                                 
 dense (Dense)               (None, 500)               19000     
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 500)               250500    
                                                   

c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


confusion_matrix:
[[494842   7802     10      0     40    113      3      2      0      0]
 [  5971 420810   1078     77   1926      0      0      0      0      0]
 [   825  65729   1111     31    345      0      0      0      0      0]
 [   149   6631    364     46     48      0      0      3      0      0]
 [  1130  52229    137     12   2031      0      0      1      0      0]
 [    97      1      0      0      0    852     69      0      0      0]
 [    38      0      0      0      0     27   5363      1      0      0]
 [    41      0      0      0      0      0      2     61      0      0]
 [    17      0      0      0      0      0      0     23      0      0]
 [     9      0      0      0      0      0      0     20      0      0]]
End of confusion_matrix:


c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98    502812
           1       0.76      0.98      0.86    429862
           2       0.41      0.02      0.03     68041
           3       0.28      0.01      0.01      7241
           4       0.46      0.04      0.07     55540
           5       0.86      0.84      0.85      1019
           6       0.99      0.99      0.99      5429
           7       0.55      0.59      0.57       104
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        29

    accuracy                           0.86   1070117
   macro avg       0.53      0.44      0.44   1070117
weighted avg       0.83      0.86      0.82   1070117

End of Classification Report:
dense_nn mc : {'accuracy': 0.8644998630990817, 'recall': 0.8644998630990817, 'precision': 0.8256241834073993, 'f1s': 0.8176462859703906, 'FPR': 0.015055570766768701, 'FNR': 0.1355001369009

...

c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


confusion_matrix:
[[495462   7229     31      2     80      5      2      0      0      0]
 [  6628 417081   2956    161   3036      0      0      0      0      0]
 [   905  64131   2710     85    211      0      0      0      0      0]
 [   139   6269    725     54     54      0      0      0      0      0]
 [  1280  50876    383      1   2999      0      0      0      0      0]
 [   167      0      0      0      0    852      0      0      0      0]
 [    52      0      0      0      0      3   5374      0      0      0]
 [   104      0      0      0      0      0      0      0      0      0]
 [    40      0      0      0      0      0      0      0      0      0]
 [    29      0      0      0      0      0      0      0      0      0]]
End of confusion_matrix:


c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98    502811
           1       0.76      0.97      0.86    429862
           2       0.40      0.04      0.07     68042
           3       0.18      0.01      0.01      7241
           4       0.47      0.05      0.10     55539
           5       0.99      0.84      0.91      1019
           6       1.00      0.99      0.99      5429
           7       0.00      0.00      0.00       104
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        29

    accuracy                           0.86   1070116
   macro avg       0.48      0.39      0.39   1070116
weighted avg       0.83      0.86      0.82   1070116

End of Classification Report:
dense_nn mc : {'accuracy': 0.8639549357265941, 'recall': 0.8639549357265941, 'precision': 0.8251896689222412, 'f1s': 0.8212345204461979, 'FPR': 0.015116118252600654, 'FNR': 0.1360450642734

c:\Users\Administrateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrateur\Desktop\GDLC\src\models\model.py:88: RuntimeWarning: invalid value encountered in longlong_scalars
  class_precision[i] = tp / (tp + fp)


In [10]:
# creating the directories if they don't exist
if not os.path.isdir('./results'):
    os.mkdir('./results')

if not os.path.isdir('./results/{}'.format(dataset_name)):
    os.mkdir('./results/{}'.format(dataset_name))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

# saving the results to a file for future refernece
filename = ('./results/{}/{}.json'.format(dataset_name,
            time.strftime("%Y%m%d-%H%M%S")))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()
